In [28]:
import pandas as pd
import numpy as np

In [29]:
df=pd.read_csv("flight_data.csv")

In [30]:
df

,Date of Booking,Date of Journey,Airline-Class,Departure Time,Arrival Time,Duration,Total Stops,Price
0,15/01/2023,16/01/2023,SpiceJet \nSG-8169\nECONOMY,20:00\nDelhi,22:05\nMumbai,02h 05m,non-stop,"5,335"
1,15/01/2023,16/01/2023,Indigo \n6E-2519\nECONOMY,23:00\nDelhi,01:20\nMumbai,02h 20m,non-stop,"5,899"
2,15/01/2023,16/01/2023,GO FIRST \nG8- 354\nECONOMY,22:30\nDelhi,00:40\nMumbai,02h 10m,non-stop,"5,801"
3,15/01/2023,16/01/2023,SpiceJet \nSG-8709\nECONOMY,18:50\nDelhi,20:55\nMumbai,02h 05m,non-stop,"5,794"
4,15/01/2023,16/01/2023,Air India \nAI-805\nECONOMY,20:00\nDelhi,22:10\nMumbai,02h 10m,non-stop,"5,955"
...,...,...,...,...,...,...,...,...
452083,15/01/2023,06/03/2023,Vistara \nUK-926\nBUSINESS,06:45\nAhmedabad,19:50\nChennai,13h 05m,1-stop,"65,028"
452084,15/01/2023,06/03/2023,Vistara \nUK-918\nBUSINESS,05:05\nAhmedabad,16:20\nChennai,11h 15m,1-stop,"69,254"
452085,15/01/2023,06/03/2023,Vistara \nUK-918\nBUSINESS,05:05\nAhmedabad,16:20\nChennai,11h 15m,1-stop,"69,254"
452086,15/01/2023,06/03/2023,Vistara \nUK-946\nBUSINESS,08:40\nAhmedabad,19:50\nChennai,11h 10m,1-stop,"72,980"


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Date of Booking  452088 non-null  object
 1   Date of Journey  452088 non-null  object
 2   Airline-Class    452088 non-null  object
 3   Departure Time   452088 non-null  object
 4   Arrival Time     452088 non-null  object
 5   Duration         452088 non-null  object
 6   Total Stops      452088 non-null  object
 7   Price            452088 non-null  object
dtypes: object(8)
memory usage: 27.6+ MB


# DATA CLEANING

In [32]:
df[['Airline', 'FlightNumber', 'Class']] = df['Airline-Class'].str.split('\n', expand=True)

In [33]:
df['Date of Booking']= pd.to_datetime(df['Date of Booking'],format='%d/%m/%Y')

In [34]:
df['Date of Journey']= pd.to_datetime(df['Date of Journey'],format='%d/%m/%Y')

In [35]:
df[['Departure Time','Departure City']] = df['Departure Time'].str.split('\n', expand=True)

In [36]:
df[['Arrival Time','Arrival City']] = df['Arrival Time'].str.split('\n', expand=True)

In [37]:
df['days_before_flight']=(df['Date of Journey']-df['Date of Booking']).dt.days

In [38]:
df['journey_day']=df['Date of Journey'].dt.day

In [39]:
df['journey_day_name']=df['Date of Journey'].dt.day_name()

In [40]:
df['Price'] = df['Price'].replace(',', '',regex=True).astype(int)

In [41]:
df['Total Stops'] = df['Total Stops'].str.replace(r'\n\s*\t*', '', regex=True)
df['Total Stops'] = df['Total Stops'].str.replace(r'(stop).*', r'\1', regex=True)

In [42]:
df['Duration'] = df['Duration'].str.extract(r'(\d+)h (\d+)m') \
                                     .astype(float) \
                                     .apply(lambda x: round(x[0] + x[1] / 60, 4), axis=1)

In [43]:
df['arrival_time'] = pd.to_datetime(df['Arrival Time'], format='%H:%M').dt.hour

In [44]:
df['arrival_category'] = df['arrival_time'].apply(lambda x: 'Before 7pm' if x < 19 else 'After 7pm')

In [45]:
df.drop(columns=['Date of Booking','Date of Journey','Airline-Class','arrival_time','FlightNumber'],inplace=True)

# Feature Engineering

In [46]:
df['Route'] = df['Departure City'] + ' -> ' + df['Arrival City']

In [47]:
df['Departure_Time_hr']=pd.to_datetime(df['Departure Time'], format='%H:%M').dt.hour
df['Arrival_Time_hr']=pd.to_datetime(df['Arrival Time'], format='%H:%M').dt.hour

In [48]:
df.drop(['Departure Time','Arrival Time'],axis=1,inplace=True)

In [49]:
df

,Duration,Total Stops,Price,Airline,Class,Departure City,Arrival City,days_before_flight,journey_day,journey_day_name,arrival_category,Route,Departure_Time_hr,Arrival_Time_hr
0,2.0833,non-stop,5335,SpiceJet,ECONOMY,Delhi,Mumbai,1,16,Monday,After 7pm,Delhi -> Mumbai,20,22
1,2.3333,non-stop,5899,Indigo,ECONOMY,Delhi,Mumbai,1,16,Monday,Before 7pm,Delhi -> Mumbai,23,1
2,2.1667,non-stop,5801,GO FIRST,ECONOMY,Delhi,Mumbai,1,16,Monday,Before 7pm,Delhi -> Mumbai,22,0
3,2.0833,non-stop,5794,SpiceJet,ECONOMY,Delhi,Mumbai,1,16,Monday,After 7pm,Delhi -> Mumbai,18,20
4,2.1667,non-stop,5955,Air India,ECONOMY,Delhi,Mumbai,1,16,Monday,After 7pm,Delhi -> Mumbai,20,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452083,13.0833,1-stop,65028,Vistara,BUSINESS,Ahmedabad,Chennai,50,6,Monday,After 7pm,Ahmedabad -> Chennai,6,19
452084,11.2500,1-stop,69254,Vistara,BUSINESS,Ahmedabad,Chennai,50,6,Monday,Before 7pm,Ahmedabad -> Chennai,5,16
452085,11.2500,1-stop,69254,Vistara,BUSINESS,Ahmedabad,Chennai,50,6,Monday,Before 7pm,Ahmedabad -> Chennai,5,16
452086,11.1667,1-stop,72980,Vistara,BUSINESS,Ahmedabad,Chennai,50,6,Monday,After 7pm,Ahmedabad -> Chennai,8,19


In [50]:
df.to_csv('Cleaned_data.csv',index=False)